In [ ]:
import sys
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, array, ArrayType, DateType
from pyspark.sql import Row, Column
import datetime
import json
import boto3
import logging
import calendar
import uuid
import time
from dateutil import relativedelta
from datetime import timedelta
import argparse

In [ ]:
####################################################################
BATCH=True
BUCKET='oracle-ligands-stats-data'
####################################################################

SUMMARY_RECORDS_COLUMNS = StructType([
    StructField("table_name", StringType(), True),
    StructField("lower_bound", IntegerType(), True),
    StructField("upper_bound", IntegerType(), True),
    StructField("status", StringType(), True),
    StructField("timestamp", StringType(), True),
    StructField("total_count", StringType(), True),
    StructField("failed_count", StringType(), True)
   ])

sc = SparkContext.getOrCreate()

if BATCH:
    spark = SparkSession(sc)

In [ ]:
path='s3://'+BUCKET+'/summary_records/*/*'
print(path)
df_output=spark.read.format("com.databricks.spark.csv").option("header", 'false').schema(SUMMARY_RECORDS_COLUMNS).option("delimiter", ',').load(path)

In [ ]:
df_output.sort('table_name', 'lower_bound').show(20,False)

In [ ]:
opath='s3://'+BUCKET+'/final_report/'
df_output.repartition(1).write.option("delimiter", ',').option("header", "false").option("quoteAll", "true").option("quote", "\"").csv(opath)
